In [1]:
%pip install langchain chromadb openai tiktoken


Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

embedding_function = OpenAIEmbeddings()

loader = CSVLoader("./data/contratos.csv", encoding="utf8")
documents = loader.load()

db = Chroma.from_documents(documents, embedding_function)
retriever = db.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=None)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(chain.invoke("Qué contratos hay en Informática?. Muestra solo el código de empleado y la fecha de contrato."))


En el departamento de Informática hay los siguientes contratos:

1. Contrato con código de empleado MBM, nombre MIGU MARQ, DNI 428307710, fecha de contrato 24/02/1992, fecha de baja 31/07/1995, actividad EO, centro OFICINA LAS PALMAS - O, matrícula 198, tipo de contrato INDEFINIDO, horas semanales 40, categoría profesional JEFE INFORMATICA.

2. Contrato con código de empleado MBM, nombre MIGU MARQ, DNI 428307710, fecha de contrato 01/08/1995, fecha de baja 01/08/2105, actividad EO, centro OFICINA LAS PALMAS - O, matrícula 48, tipo de contrato INDEFINIDO, horas semanales 40, categoría profesional JEFE INFORMATICA.


In [10]:
print(chain.invoke("Cuantos empleados distintos hay por cada departamento?"))

Basado en el contexto proporcionado, todos los documentos se refieren al mismo empleado, PEDR DUQU, con el código de empleado DAP, en el departamento de ADMINISTRACION. No hay información sobre otros empleados en otros departamentos.

Por lo tanto, hay **1 empleado** en el departamento de **ADMINISTRACION**.
